### Пошук імен політиків серед сутностей, що раніше були знайдені NER-моделлю

#### Визначення сутностей у розбитих на речення новинах у репозиторії з експертами у ноутбуці 1_Names_and_Keywords_search.ipynb

In [1]:
import pandas as pd
import re

In [2]:
entities_filepath = '../data/summer_entities.csv'
entities = pd.read_csv(entities_filepath, index_col=[0])
entities.columns

Index(['link', 'language', 'all_ent_str', 'name_and_kw', 'kw_and_ent',
       'names_and_kw_str', 'names_str', 'names_sets_str',
       'one_name_per_set_str', 'string_names_sets_str', 'filt_kw_names_str',
       'checked_with_conllu', 'checked_names_str', 'kw_names_unique_str',
       'found', 'found_names_str', 'dict_str', 'all_found_names_str',
       'all_dict_str'],
      dtype='object')

In [3]:
news_filepath = '../data/summer.csv'
news = pd.read_csv(news_filepath, index_col=[0])
news.columns

/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'pub_type', 'author', 'tags',
       'source', 'author_title', 'domain_alias', 'mycategory',
       'found_names_str', 'author_gender', 'found_expert_names_str',
       'found_not_expert_names_str'],
      dtype='object')

#### Сутності вже були знайдені і збережені у вигляді рядків у стовпчику all_ent_str

In [4]:
news['all_ent_str'] = entities['all_ent_str']
del entities

**Сутності складаються з таких частин:**
 - token_range: (7, 10)
 - type: LOC
 - text: окупованому Росією Криму
 - score: 0.54
 
**Частини розділені між собою через <§§>, сутності - через <;;>, а речення - через <@@>**

In [32]:
news.all_ent_str.iloc[0]

'(7, 10)<§§>LOC<§§>окупованому Росією Криму<§§>0.54<@@>(1, 2)<§§>LOC<§§>Страсбурзі<§§>0.79<;;>(8, 12)<§§>ORG<§§>Парламентська асамблея Ради Європи<§§>0.31<@@><@@><@@><@@><@@><@@>(9, 11)<§§>PERS<§§>Марією Мезенцевою<§§>0.99<;;>(14, 15)<§§>LOC<§§>Страсбурга<§§>0.99<@@>(1, 2)<§§>PERS<§§>Гончаренка<§§>0.87<;;>(5, 6)<§§>ORG<§§>ПАРЄ<§§>0.47<@@><@@>(4, 5)<§§>LOC<§§>Ісландії<§§>0.52<;;>(5, 7)<§§>PERS<§§>Сунна Торхілдур<§§>0.33<@@>(10, 11)<§§>LOC<§§>Криму<§§>0.97<@@>(3, 4)<§§>LOC<§§>Росію<§§>0.46<@@>(3, 4)<§§>LOC<§§>Криму<§§>0.77<@@><@@>(11, 13)<§§>ORG<§§>телеканалу ATR<§§>0.68<@@><@@>(2, 4)<§§>PERS<§§>Мустафі Джемілєву<§§>0.83<;;>(5, 7)<§§>PERS<§§>Рефату Чубарову<§§>0.61<;;>(9, 10)<§§>LOC<§§>Крим<§§>1.06<@@><@@>(0, 1)<§§>LOC<§§>Росія<§§>0.76<@@><@@><@@>(7, 8)<§§>LOC<§§>Страсбурзі<§§>0.89<@@><@@>(4, 5)<§§>LOC<§§>Донбасі<§§>0.84<@@>(9, 10)<§§>LOC<§§>Білорусі<§§>0.75<@@><@@><@@><@@>(4, 6)<§§>PERS<§§>Олексія Навального<§§>0.79<@@><@@>(0, 4)<§§>ORG<§§>Парламентська асамблея Ради Європи<§§>0.33<;;>(

In [6]:
ukr_politician_dict_path = 'dicts/politicians_ukr.csv'
ru_politician_dict_path = 'dicts/politicians_ru.csv'

ukr_names_filepath = 'dicts/names_ukr.txt'
ru_names_filepath = 'dicts/names_ru.txt'

politicians_dict_ukr = pd.read_csv(ukr_politician_dict_path, sep=';')
politicians_dict_ru = pd.read_csv(ru_politician_dict_path, sep=';')

In [7]:
len(politicians_dict_ru.surname.values)

67

#### Додаємо різні форми імен для пошуку серед сутностей:
 - ім'я+прізвище
 - ім'я+по-батькові
 - прізвище

In [33]:
def form_fullname_pattern(politician):
    return rf"({politician['name']} {politician['surname']})"

In [34]:
def form_name_patronim_pattern(politician):
#     return rf"((?<!{politician['surname']} ){politician['name']} {politician['patronim']}(?! {politician['surname']}))"
    return rf"({politician['name']} {politician['patronim']})"
    

 **Також додаємо форми неправильних імен, коли знайдено прізвище таке, як у політика, але ім'я перед ним невірне**
 
 Таким чином обробляємо випадки людей з однаковими прізвищами. Щоб такі імена як, Олена Зеленська, Олег Ляшко і Оксана Марченко не рахувалися як згадки Володимира Зеленського, Віктора Ляшка та Сергія Марченка відповідно.

In [35]:
def form_wrong_name_pattern(politician, ending, all_names):
    wrong_names = [n for n in all_names if n != politician['shortname']] 
    wrong_names = '|'.join(wrong_names)
    wrong_names_forms = rf"(({wrong_names}){ending} {politician['surname']})"
    return wrong_names_forms

In [36]:
ending_ukr = r"[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b"
ending_ru = r"[аийуыяеоюёьэмфцвскпнртгшлдщзбжх]{,3}\b"

with open(ukr_names_filepath) as f:
    all_names_ukr = [n.strip() for n in f.readlines()]

with open(ru_names_filepath) as f:
    all_names_ru = [n.strip() for n in f.readlines()]

def add_endings(name_dict, ending, all_names):
    name_forms_dict = name_dict[['surname','name','patronim']].applymap(lambda x: x+ending if pd.notnull(x) else None)
    name_forms_dict['fullname'] = name_dict.fullname
    name_forms_dict['shortname'] = name_dict['name']
    name_forms_dict['name_forms'] = name_forms_dict.apply(form_fullname_pattern, axis=1)
    name_forms_dict['wrong_names'] = name_forms_dict.apply(form_wrong_name_pattern, ending=ending, all_names=all_names,axis=1)
    name_forms_dict['name_patronim'] = name_forms_dict.apply(form_name_patronim_pattern, axis=1)
    return name_forms_dict

    
name_forms_ukr = add_endings(politicians_dict_ukr, ending_ukr, all_names_ukr)
name_forms_ru = add_endings(politicians_dict_ru, ending_ru, all_names_ru)


In [37]:
name_forms_ukr.head()

,surname,name,patronim,fullname,shortname,name_forms,wrong_names,name_patronim
0,"Зеленськ[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Володимир[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b",Олександрович[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]...,Володимир Зеленський,Володимир,"(Володимир[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3...",((Анатолі|Андрі|Антон|Аркаді|Арсен|Артем|Артур...,"(Володимир[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3..."
1,"Порошенк[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Петр[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Олексійович[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,...",Петро Порошенко,Петр,"(Петр[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b П...",((Анатолі|Андрі|Антон|Аркаді|Арсен|Артем|Артур...,"(Петр[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b О..."
2,Венед(і|и)ктов[аийуяіеоюєїьмфцвскпнртгшлдщзбжх...,"Ірин[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Валентинівн[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,...",Ірина Венедіктова,Ірин,"(Ірин[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b В...",((Анатолі|Андрі|Антон|Аркаді|Арсен|Артем|Артур...,"(Ірин[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b В..."
3,"Аваков[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Арсен[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Борисович[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b",Арсен Аваков,Арсен,"(Арсен[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b ...",((Анатолі|Андрі|Антон|Аркаді|Артем|Артур|Богда...,"(Арсен[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b ..."
4,"Степанов[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b","Максим[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b",Володимирович[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]...,Максим Степанов,Максим,"(Максим[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b...",((Анатолі|Андрі|Антон|Аркаді|Арсен|Артем|Артур...,"(Максим[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\b..."


#### Об'єднуємо регулярки українською та російською мовами в один датафрейм

In [38]:
name_forms_ru = name_forms_ru.set_index('fullname', drop=False)
name_forms_ukr = name_forms_ukr.set_index('fullname', drop=False)

In [39]:
name_forms_all = name_forms_ukr.copy()
name_forms_all['forms_ukr_ru'] = name_forms_ukr.name_forms+'|'+name_forms_ru.name_forms
name_forms_all.forms_ukr_ru.iloc[0]

'(Володимир[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\\b Зеленськ[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\\b)|(Владимир[аийуыяеоюёьэмфцвскпнртгшлдщзбжх]{,3}\\b Зеленск[аийуыяеоюёьэмфцвскпнртгшлдщзбжх]{,3}\\b)'

In [40]:
name_forms_all['surname_ukr_ru'] = '('+name_forms_ukr.surname+'|'+name_forms_ru.surname+')'
name_forms_all.surname_ukr_ru.iloc[0]

'(Зеленськ[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\\b|Зеленск[аийуыяеоюёьэмфцвскпнртгшлдщзбжх]{,3}\\b)'

In [41]:
name_forms_all['wrong_forms_ukr_ru'] = name_forms_ukr.wrong_names+'|'+name_forms_ru.wrong_names
name_forms_all.wrong_forms_ukr_ru.iloc[0]

"((Анатолі|Андрі|Антон|Аркаді|Арсен|Артем|Артур|Богдан|Борис|Валентин|Валері|Васил|Вадим|Віктор|Віталі|Владислав|Всеволод|В'ячеслав|Генн?аді|Георгі|Гліб|Гнат|Григорі|Давид|Данил|Денис|Дмитр|Єгор|Євген|Захар|Зорян|Ілл|Іван|Ігор|Кирил|Костянтин|Леонід|Лук'ян|Любомир|Маркіян|Макар|Максим|Марк|Мартин|Микит|Микол|Мирон|Мирослав|Михайл|Назар|Олег|Олександр|Олексі|Олес|Остап|Павл|Пантелеймон|Панас|Петр|Руслан|Роман|Ростислав|Семен|Святослав|Сергі|Степан|Тарас|Тимофі|Федір|Юрі|Юхим|Яків|Ярослав|Алін|Аліс|Алл|Альбін|Альон|Анастасі|Ангелін|Анжелік|Анн|Антонін|Василин|Веронік|Вікторі|Галин|Ганн|Дарин|Дарі|Евелін|Євдокі|Єлизавет|Злат|Іванн|Ілон|Інн|Карин|Катерин|Ірин|Євгені|Ксені|Ларис|Ліді|Лілі|Лін|Лєн|Любов|Людмил|Маргарит|Марин|Марі|Март|Мирослав|Наді|Натал|Оксан|Олен|Ольг|Полін|Римм|Саш|Світлан|Соломі|Софі|Таїсі|Тетян|Тамар|Мелан|Єкатерин|Роксолан|Христин|Юлі|Ян|Янін|Сніжан|Мелінд|Ірен|Марічк|Крістін|Устин|Кат|Наст|Люд|Люб|Даш|Ліз|Тон|Тол|Слав|Наташ|Діан|Центр|центр|Маршал)[аийуяіеоюєїьмфцвскп

In [42]:
name_forms_all['name_patronim_ukr_ru'] = name_forms_ukr.name_patronim+'|'+name_forms_ru.name_patronim
name_forms_all.name_patronim_ukr_ru.iloc[0]

'(Володимир[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\\b Олександрович[аийуяіеоюєїьмфцвскпнртгшлдщзбжх]{,3}\\b)|(Владимир[аийуыяеоюёьэмфцвскпнртгшлдщзбжх]{,3}\\b Александрович[аийуыяеоюёьэмфцвскпнртгшлдщзбжх]{,3}\\b)'

In [43]:
mentions = news[['link', 'language']].copy()

#### Шукаємо у рядку all_ent_str варіанти імен політиків, рахуємо кількість входжень кожної окремої форми (повного імені, прізвища, по-батькові та неправильних імен)

In [44]:
def search_for_name(entities, entities_col, name_forms, politician, col_suffix, mentions):
    res = entities[entities_col].str.extractall(name_forms)
    ment_col = politician + col_suffix
    mentions[ment_col] = res.groupby(level=[0]).size()
    return mentions

In [28]:
for i, r in name_forms_all.iterrows():
    mentions = search_for_name(news, 'all_ent_str', r.forms_ukr_ru, i, '_fullname', mentions)

In [ ]:
for i, r in name_forms_all.iterrows():
    mentions = search_for_name(news, 'all_ent_str', r.surname_ukr_ru, i, '_surname', mentions)

In [ ]:
for i, r in name_forms_all.iterrows():
    mentions = search_for_name(news, 'all_ent_str', r.wrong_forms_ukr_ru, i, '_wrong', mentions)

In [ ]:
for i, r in name_forms_all.iterrows():
    if pd.notna(r.patronim):
        mentions = search_for_name(news, 'all_ent_str', r.name_patronim_ukr_ru, i, '_patronim', mentions)

In [84]:
mentions

,link,language,Володимир Зеленський_fullname,Петро Порошенко_fullname,Ірина Венедіктова_fullname,Арсен Аваков_fullname,Максим Степанов_fullname,Юлія Тимошенко_fullname,Денис Шмигаль_fullname,Давид Арахамія_fullname,...,Михайло Федоров_patronim,Микола Тищенко_patronim,Ольга Стефанішина_patronim,Кирило Шевченко_patronim,Тарас Козак_patronim,Олександр Дубінський_patronim,Олег Уруський_patronim,Олександр Качура_patronim,Ярослав Железняк_patronim,Андрій Деркач_patronim
id,,,,,,,,,,,,,,,,,,,,,
2075213,https://news.obozrevatel.com/ukr/abroad/italiy...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2077435,https://news.obozrevatel.com/ukr/auto/news/cup...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2076296,https://news.obozrevatel.com/ukr/travel/dvi-uk...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2081964,https://www.rbc.ua/ukr/news/ukraine-hotyat-soz...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2081965,https://www.rbc.ua/ukr/news/razvedchiki-morsko...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072553,https://www.pravda.com.ua/news/2021/05/11/7293...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2072554,https://www.pravda.com.ua/news/2021/05/11/7293...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2072555,https://www.pravda.com.ua/news/2021/05/12/7293...,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Для політиків без по-батькові присвоюємо 0 у відповідні стовпчики

In [43]:
for pol in politicians_dict_ukr.fullname.values:
    patronim_col = pol+'_patronim'
    if patronim_col not in mentions.columns:
        mentions[patronim_col] = 0

#### Рахуємо сумарну кількість згадок політика у новині за таким принципом:
 - якщо в новині немає неправильного варіанту, то додаємо кіл-ть входжень прізвища та ім'я+по-батькові
 - якщо в новині були згадки неправильного імені, то згадки просто прізвища рахувати ненадійно, сумуються тільки згадки повного імені (ім'я+прізвище) та ім'я+по-батькові

In [44]:
for pol in politicians_dict_ukr.fullname.values:
    patronim_col = pol+'_patronim'
    surname_col = pol+'_surname'
    fullname_col = pol+'_fullname'
    wrong_name_col = pol+'_wrong'
    res_col = pol+'_res'
    mentions[res_col] = 0
    mask1 = (mentions[wrong_name_col]==0)&(mentions[surname_col]>0)
    mentions.loc[mask1, res_col] = mentions[mask1][surname_col] + mentions[mask1][patronim_col]
    
    mask2 = (mentions[wrong_name_col]>0)&(mentions[surname_col]>0)
    mentions.loc[mask2, res_col] = mentions[mask2][fullname_col] + mentions[mask2][patronim_col]
    

In [53]:
res_cols = [col for col in mentions.columns if '_res' in col]
res_cols

['Володимир Зеленський_res',
 'Петро Порошенко_res',
 'Ірина Венедіктова_res',
 'Арсен Аваков_res',
 'Максим Степанов_res',
 'Юлія Тимошенко_res',
 'Денис Шмигаль_res',
 'Давид Арахамія_res',
 'Віктор Медведчук_res',
 'Міхеіл Саакашвілі_res',
 'Вадим Рабінович_res',
 'Олексій Гончаренко_res',
 'Віталій Кличко_res',
 'Юрій Бойко_res',
 'Дмитро Разумков_res',
 'Віктор Ляшко_res',
 'Андрій Єрмак_res',
 'Гео Лерос_res',
 'Іван Баканов_res',
 'Артем Ситник_res',
 'Назар Холодницький_res',
 'Владислав Криклій_res',
 'Олександр Ткаченко_res',
 'Геннадій Труханов_res',
 'Андрій Садовий_res',
 'Борис Філатов_res',
 'Денис Малюська_res',
 'Олександр Корнієнко_res',
 'Кіра Рудик_res',
 'Дмитро Кулеба_res',
 'Олексій Данілов_res',
 'Леонід Кравчук_res',
 'Юрій Вітренко_res',
 'Антон Геращенко_res',
 'Сергій Марченко_res',
 'Олексій Резніков_res',
 'Олег Татаров_res',
 'Євгенія Кравчук_res',
 'Кирило Тимошенко_res',
 'Ігор Терехов_res',
 'Сергій Шкарлет_res',
 'Марина Лазебна_res',
 'Олексій Аресто

#### Отримані сумарні кількості згадок перетворюємо в рядки вигляду Ім'я+кіл-ть згадок і об'єднуємо їх через §

Наприклад, "Володимир Зеленський+1§Петро Порошенко+1" - у новині по разу згадувалися Зеленський і Порошенко

In [54]:
def get_mentions_dict(row):
    mentions_num = []
    for pol in politicians_dict_ukr.fullname.values:
        res_col = pol + '_res'
        if row[res_col] > 0:
            mentions_num.append('+'.join([pol, str(row[res_col])]))
    return '§'.join(mentions_num)

In [55]:
mentions['mentions_str'] = mentions.apply(lambda row: get_mentions_dict(row), axis=1)

In [75]:
mentions_filepath = 'summer_politicians_mentions.csv'
mentions.to_csv(mentions_filepath)

In [67]:
news['mentions'] = mentions['mentions_str']

#### Зберегти у файл, тільки не забути виключити перед цим зайві стовпчики

In [45]:
news.to_csv(news_filepath)